![alt text](https://)## Imports

<br>
<br>
<br>
<br>



In [ ]:
"""
Created on Tue Oct 15 00:14:08 2019

@author: anil
"""

import pandas as pd
import numpy as np
     
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize 
!pip install krovetzstemmer
import krovetzstemmer
import nltk
nltk.download("popular")
from nltk.corpus import stopwords
from krovetzstemmer import Stemmer
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")
!pip install -q keras
import keras
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import Sequential
from keras import layers

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

Using TensorFlow backend.


cu80


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
olid_training=pd.read_csv("/content/drive/My Drive/OLIDv1.0/olid-training-v1.0.tsv", sep="\t")
labels_a=pd.read_csv("/content/drive/My Drive/OLIDv1.0/labels-levela.csv", header=None)
labels_b=pd.read_csv("/content/drive/My Drive/OLIDv1.0/labels-levelb.csv")
labels_c=pd.read_csv("/content/drive/My Drive/OLIDv1.0/labels-levelc.csv")
test_a=pd.read_csv("/content/drive/My Drive/OLIDv1.0/testset-levela.tsv",sep = "\t" )
test_b=pd.read_csv("/content/drive/My Drive/OLIDv1.0/testset-levelb.tsv" ,sep = "\t")
test_c=pd.read_csv("/content/drive/My Drive/OLIDv1.0/testset-levelc.tsv",sep = "\t")
import gensim
# Load Google's pre-trained Word2Vec model.
w2v_model = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/GoogleNews-vectors-negative300.bin.gz", binary=True)

NameError: ignored

In [ ]:
X_train=olid_training[["id","tweet","subtask_a"]] # I will delete subtask_a later in the training part.
X_test=test_a
y_train=olid_training["subtask_a"]
y_test=labels_a

## Reprocessing the Data
<br>
<br>
<br>
<br>

In [ ]:
#1- Make all lowercase.

print("Lowercase operation begins ! ")


X_train.tweet = X_train.tweet.str.lower()
X_test.tweet = X_test.tweet.str.lower()

print("Lowercase operation is done ! ")


#For train data
filtered_tweets=[]
for tweet in X_train["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if ( w.isalpha()==True)] 
    filtered_tweets.append(filtered_sentence)
  
X_train["tweet_initial"]=filtered_tweets



#For test data initial tweets
filtered_tweets=[]
for tweet in X_test["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if ((w.isalpha()==True))] 
      
    filtered_tweets.append(filtered_sentence)

X_test["tweet_initial"]=filtered_tweets

#for training data initial tweets

#2-Stop word removal

print("Stop word  and punctuation removal begins !")

stop_words = set(stopwords.words('english')) 

#For train data
filtered_tweets=[]
for tweet in X_train["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if ((not w in stop_words) and w.isalpha()==True)] 
      
    filtered_tweets.append(filtered_sentence)

X_train["tweet_after_stopword"]=filtered_tweets

#For test data
filtered_tweets=[]
for tweet in X_test["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if ((not w in stop_words) and w.isalpha()==True)] 
      
    filtered_tweets.append(filtered_sentence)

X_test["tweet_after_stopword"]=filtered_tweets

print("Stop word and punctuation removal is done !")

###################################################################333
print("Stemming begins !")

stemmer = Stemmer()
stemmer.stem('utilities')  # got: 'utility'

#For stemming tranining data
dum=X_train["tweet_after_stopword"]
stemmed_tweets=[]
for tweet in dum:
    tweet_arr=[]
    for word in tweet :
        #print(stemmer.stem(word))
        tweet_arr.append(stemmer.stem(word))
    stemmed_tweets.append(tweet_arr)
    
X_train["tweet_after_stemmed"]=stemmed_tweets

#For stemming Test data
dum=X_test["tweet_after_stopword"]
stemmed_tweets=[]
for tweet in dum:
    tweet_arr=[]
    for word in tweet :
        #print(stemmer.stem(word))
        tweet_arr.append(stemmer.stem(word))
    stemmed_tweets.append(tweet_arr)
    
X_test["tweet_after_stemmed"]=stemmed_tweets
print("Stemming Done")
print("Reprocessing Done!")


Lowercase operation begins ! 
Lowercase operation is done ! 
Stop word  and punctuation removal begins !
Stop word and punctuation removal is done !
Stemming begins !
Stemming Done
Reprocessing Done!


In [ ]:
X_train.head(2)

,id,tweet,subtask_a,tweet_initial,tweet_after_stopword,tweet_after_stemmed
0,86426,@user she should ask a few native americans wh...,OFF,"[user, she, should, ask, a, few, native, ameri...","[user, ask, native, americans, take]","[user, ask, native, america, take]"
1,90194,@user @user go home you’re drunk!!! @user #mag...,OFF,"[user, user, go, home, you, re, drunk, user, m...","[user, user, go, home, drunk, user, maga, url]","[user, user, go, home, drunk, user, maga, url]"


In [ ]:
X_test.head(2)

,id,tweet,tweet_initial,tweet_after_stopword,tweet_after_stemmed
0,15923,#whoisq #wherestheserver #dumpnike #declasfisa...,"[whoisq, wherestheserver, dumpnike, declasfisa...","[whoisq, wherestheserver, dumpnike, declasfisa...","[whoisq, wherestheserver, dumpnike, declasfisa..."
1,27014,"#constitutionday is revered by conservatives, ...","[constitutionday, is, revered, by, conservativ...","[constitutionday, revered, conservatives, hate...","[constitutionday, revere, conservative, hate, ..."


## ML Baseline : Naive Bayes
<br>
<br>
<br>
<br>

In [ ]:
print("ML models begins for Task1 !!")
print("Naive Bayes begins for Task 1")

# First create bag of words for both classes (off or not off)
from collections import Counter
# Create bag of words for class 1 in subtask-a
bow_class_1=X_train[X_train["subtask_a"]=="OFF"]
bow_class_2=X_train[X_train["subtask_a"]=="NOT"]
#####################################################3
bow_off_initial= Counter([word for tweets in bow_class_1["tweet_initial"] for word in tweets])
bow_not_initial= Counter([word for tweets in bow_class_2["tweet_initial"] for word in tweets])
########################################################

bow_off_stopword= Counter([word for tweets in bow_class_1["tweet_after_stopword"] for word in tweets])
bow_not_stopword= Counter([word for tweets in bow_class_2["tweet_after_stopword"] for word in tweets])

print("Bag of word for stopword removed tweets are created !")
bow_off_stemmed = Counter([word for tweets in bow_class_1["tweet_after_stemmed"] for word in tweets])
bow_not_stemmed = Counter([word for tweets in bow_class_2["tweet_after_stemmed"] for word in tweets])
print("Bag of word for stemmed tweets are created !")

print("Naive Bayes training starting...")

# Find vocab size and total word size
vocab_off_initial=len(bow_off_initial)
vocab_not_initial=len(bow_not_initial)

vocab_off_stemmed=len(bow_off_stemmed)
vocab_not_stemmed=len(bow_not_stemmed)

vocab_off_before_stemmed=len(bow_off_stopword)
vocab_not_before_stemmed=len(bow_not_stopword)

#################################33 for initial tweets

total_count_off_initial=sum(bow_off_initial.values())
total_count_not_initial=sum(bow_not_initial.values())

#####################################


print("Vocabulary size of stemmed data class OFF is :",vocab_off_stemmed)
print("Vocabulary size of stemmed data class NOT is :",vocab_not_stemmed)

total_count_off_stemmed =sum(bow_off_stemmed.values())
total_count_not_stemmed =sum(bow_not_stemmed.values())

total_count_off_before_stemmed =sum(bow_off_stopword.values())
total_count_not_before_stemmed =sum(bow_not_stopword.values())

print("Total occurence of words in stemmed data class OFF is :",total_count_off_stemmed)
print("Total occurence of words instemmed data class NOT is :",total_count_not_stemmed)
print("Naive Bayes Training is Done!")

#TO DO'S -> HASTHAGLERIN TEMIZLENMESI LAZIM , #USER BILGISI
#test_a_copy

print("Naive Bayes Testing starting...")

# p(c|i)=p(i|c)*p(c)
# (i belonging class c ) = product( p of a test word j in class c ) * p of class c

#class probabilities

def NBClassifier(data,bow1,bow2,total_count_off,total_count_not,vocab_off,vocab_not): #parameters are important according to the vocab size , vocab count along the classes
    predicted_labels=[]
    p_off= len(X_train[X_train["subtask_a"]=="OFF"]) /len(X_train) 
    p_not= len(X_train[X_train["subtask_a"]=="NOT"]) /len(X_train) 
    for tweets in data: # i'th test instances
        # every tweet has j word
        # prob of test word j in  in class OFF
        
        result_prob_class_off=1
        result_prob_class_not=1
        for word in tweets:
            #apply laplace smooting as well
            #prob of word in class off
            
            #counts of word "j" in class c  + 1 / counts of word in class c + |v| + 1 -> 
            x1=bow1.get(word)
            if x1 == None:
                x1=0
            result_prob_class_off= result_prob_class_off* ((x1 + 1 )/ (total_count_off + vocab_off + 1))
            #prob off word in class not
            #print(result_prob_class_off)
            x2=bow2.get(word)
            if x2 == None:
                x2=0
            
            result_prob_class_not= result_prob_class_not* ((x2 + 1 )/ (total_count_not + vocab_not + 1))
            #print(result_prob_class_not)
        #Resulting probabilities , than we compare it to five a result
        result_prob_class_off=result_prob_class_off*p_off
        result_prob_class_not=result_prob_class_not*p_not
        
        if result_prob_class_off >  result_prob_class_not:
            predicted_labels.append("OFF")
        else:
            predicted_labels.append("NOT")
        
        
    print ("Naive Bayes Test is completed...")
    
    
    evaluate=X_test[["id","tweet"]]
    evaluate["Predicted_Labels"]=predicted_labels
    evaluate["Actual_Labels"]=labels_a[1]
    
    predict_arr = [1 if i == "OFF" else 0 for i in evaluate["Predicted_Labels"]]
    actual_arr = [1 if i == "OFF" else 0 for i in evaluate["Actual_Labels"]]
       # print("ex1 prob of belong class off",result_prob_class_off)
        
        #print("ex1 prob of belong class not",result_prob_class_not)
        
        
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix
    
    a=classification_report(actual_arr, predict_arr)
    b=confusion_matrix(actual_arr, predict_arr)
    return a,b

res1,re1=NBClassifier(X_test["tweet_after_stemmed"],bow_off_stemmed,bow_not_stemmed,total_count_off_stemmed,total_count_not_stemmed,vocab_off_stemmed,vocab_not_stemmed)
res2,re2=NBClassifier(X_test["tweet_after_stopword"],bow_off_stopword,bow_not_stopword,total_count_off_before_stemmed,total_count_not_before_stemmed,vocab_off_before_stemmed,vocab_not_before_stemmed)
res3,re3=NBClassifier(X_test["tweet_initial"],bow_off_initial,bow_not_initial,total_count_off_initial,total_count_not_initial,vocab_off_initial,vocab_not_initial)
print("Naive Bayes Confusion Matrix on only tokenizing for offensEval Task 1")
print(re3)
print(res3)



print("Naive Bayes Confusion Matrix after Stopword Removal for offensEval Task 2")
print(re2)
print(res2)

print("Naive BayesConfusion Matrix after Stopword Removal + Stemming for offensEval Task 2")

print(re1)
print(res1)

ML models begins for Task1 !!
Naive Bayes begins for Task 1
Bag of word for stopword removed tweets are created !
Bag of word for stemmed tweets are created !
Naive Bayes training starting...
Vocabulary size of stemmed data class OFF is : 7845
Vocabulary size of stemmed data class NOT is : 11455
Total occurence of words in stemmed data class OFF is : 57600
Total occurence of words instemmed data class NOT is : 108903
Naive Bayes Training is Done!
Naive Bayes Testing starting...
Naive Bayes Test is completed...
Naive Bayes Test is completed...
Naive Bayes Test is completed...
Naive Bayes Confusion Matrix on only tokenizing for offensEval Task 1
[[359 261]
 [ 46 194]]
              precision    recall  f1-score   support

           0       0.89      0.58      0.70       620
           1       0.43      0.81      0.56       240

    accuracy                           0.64       860
   macro avg       0.66      0.69      0.63       860
weighted avg       0.76      0.64      0.66       860

In [ ]:
#confusion_matrix(actual_arr, predict_arr)



![alt text](https://)## More Data Reprocessing before CNN + LSTM

<br>
<br>
<br>
<br>

In [ ]:
#TEST Ya
print(labels_a.shape)
labels_a.head(1)

(860, 2)


,0,1
0,15923,OFF


In [ ]:
# I did some reprocessing again and make data non-tokenize again

#some reverse-reprocessingg due to i am decide to use Keras-tokenizer.
m=[]
for tweet in X_train["tweet_after_stemmed"]:
    a=" ".join(tweet)
    m.append(a)
X_train["tweet_stemmed"]=m
n=[]
for tweet in X_train["tweet_after_stopword"]:
    b=" ".join(tweet)
    n.append(b)
X_train["tweet_stopword"]=n

z=[]
for tweet in X_train["tweet_initial"]:
    b=" ".join(tweet)
    z.append(b)
X_train["tweet_initial_nontoken"]=z



m=[]
for tweet in X_test["tweet_after_stemmed"]:
    c=" ".join(tweet)
    m.append(c)
X_test["tweet_stemmed"]=m
n=[]
for tweet in X_test["tweet_after_stopword"]:
    d=" ".join(tweet)
    n.append(d)
X_test["tweet_stopword"]=n

z=[]
for tweet in X_test["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_test["tweet_initial_nontoken"]=z

In [ ]:
X_train.head(1)

,id,tweet,subtask_a,tweet_initial,tweet_after_stopword,tweet_after_stemmed,tweet_stemmed,tweet_stopword,tweet_initial_nontoken
0,86426,@user she should ask a few native americans wh...,OFF,"[user, she, should, ask, a, few, native, ameri...","[user, ask, native, americans, take]","[user, ask, native, america, take]",user ask native america take,user ask native americans take,user she should ask a few native americans wha...


In [ ]:
X_test.head(1)

,id,tweet,tweet_initial,tweet_after_stopword,tweet_after_stemmed,tweet_stemmed,tweet_stopword,tweet_initial_nontoken
0,15923,#whoisq #wherestheserver #dumpnike #declasfisa...,"[whoisq, wherestheserver, dumpnike, declasfisa...","[whoisq, wherestheserver, dumpnike, declasfisa...","[whoisq, wherestheserver, dumpnike, declasfisa...",whoisq wherestheserver dumpnike declasfisa dem...,whoisq wherestheserver dumpnike declasfisa dem...,whoisq wherestheserver dumpnike declasfisa dem...


*italicized text*## Word Embedding
<br>
<br>
<br>
<br>

In [ ]:
print("Word Embedding Starts... for stemmed tweets")
# train your word embeddings during the training of your neural network. 
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train["tweet_initial_nontoken"])
X_train_initial = tokenizer.texts_to_sequences(X_train["tweet_initial_nontoken"])
X_test_initial = tokenizer.texts_to_sequences(X_test["tweet_initial_nontoken"])
vocab_size_initial = len(tokenizer.word_index) + 1 

tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train["tweet_stopword"])
X_train_stopword=tokenizer.texts_to_sequences(X_train["tweet_stopword"])
X_test_stopword=tokenizer.texts_to_sequences(X_test["tweet_stopword"])
vocab_size_stopword=  len(tokenizer.word_index) + 1 

tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train["tweet_stemmed"])
X_train_stemmed=tokenizer.texts_to_sequences(X_train["tweet_stemmed"])
X_test_stemmed=tokenizer.texts_to_sequences(X_test["tweet_stemmed"])
vocab_size_stemmed = len(tokenizer.word_index) + 1 


y_train=[1 if i == "OFF" else 0 for i in X_train["subtask_a"]]
y_test= [1 if i == "OFF" else 0 for i in labels_a[1]]



print(X_train["tweet_initial_nontoken"][10000])
print(X_train_initial[10000])
print(X_train["tweet_stopword"][10000])
print(X_train_stopword[1])


Word Embedding Starts... for stemmed tweets
user user real conservatives are stepping up
[1, 1, 208, 48, 10, 3738, 61]
user user real conservatives stepping
[1, 1, 23, 205, 824, 1, 8, 2]


In [ ]:
#implement padding

from keras.preprocessing.sequence import pad_sequences
max_len = 100
X_train_initial = pad_sequences(X_train_initial, padding='post', maxlen=max_len)
X_test_initial = pad_sequences(X_test_initial, padding='post', maxlen=max_len)

X_train_stopword=pad_sequences(X_train_stopword, padding='post', maxlen=max_len)
X_test_stopword=pad_sequences(X_test_stopword, padding='post', maxlen=max_len)

X_train_stemmed=pad_sequences(X_train_stemmed, padding='post', maxlen=max_len)
X_test_stemmed=pad_sequences(X_test_stemmed, padding='post', maxlen=max_len)

print(X_train["tweet"][30])
print(X_train_initial[30])

print(X_train["tweet_stopword"][30])
print(X_train_stopword[30])

print(X_train["tweet_stemmed"][30])
print(X_train_stemmed[30])



@user glad to see your friends are supporting metis rights as much as you are supporting zionists. url
[   1  453    4   92   32  564   10  546 8649  284   37  116   37    5
   10  546 6125   17    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
user glad see friends supporting metis rights much supporting zionists url
[   1  335   25  444  427 8500  179   37  427 5978    2    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    

### Keras Metrics

<br>
<br>
<br>
<br>

In [ ]:
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report
def recall_m(true_Y, pred_Y):
        TP = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        possible_pos = K.sum(K.round(K.clip(true_Y, 0, 1)))
        rec = TP / (possible_pos + K.epsilon())
        return rec

def precision_m(true_Y, pred_Y):
        true_positives = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(pred_Y, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(true_Y, pred_Y):
    pres = precision_m(true_Y, pred_Y)
    rec = recall_m(true_Y, pred_Y)
    return 2*((pres*rec)/(pres+rec+K.epsilon()))

from keras.models import Sequential
from keras import layers


## Conv Net with Embedding learned as part of  model
<br>
<br>
<br>
<br>




In [ ]:
def convnet_embedding_layer(vocab_size,X_train,X_test,y_train,y_test):

  print("ConvNet Begins...")
  embedding_dim = 100
  early_stopping = [EarlyStopping(monitor='val_acc',
                          min_delta=0,restore_best_weights=True,
                          patience=50,
                          verbose=1, mode='auto')]
  model = Sequential()
  model.add(layers.Embedding(vocab_size, embedding_dim, input_length=max_len))
  model.add(layers.Conv1D(128, 3, activation='relu'))
  model.add(layers.GlobalMaxPooling1D())
  model.add(layers.Dense(100, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['acc',f1_m,precision_m, recall_m])
  model.summary()

  history = model.fit(X_train, y_train,callbacks=early_stopping,
                      epochs=150,
                      verbose=1,
                      validation_split=0.4,
                      batch_size=10)
  loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, y_train, verbose=1)
  print("CNN Training Loss: {:.4f}".format(loss))
  print("CNN Training Accuracy: {:.4f}".format(accuracy))
  print("CNN Training f1 score: {:.4f}".format(f1_score))
  print("CNN Training Precision: {:.4f}".format(precision))
  print("CNN Training Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)
  print("CNN Test Loss: {:.4f}".format(loss))
  print("CNN Test Accuracy: {:.4f}".format(accuracy))
  print("CNN Test f1 score: {:.4f}".format(f1_score))
  print("CNN Test Precision: {:.4f}".format(precision))
  print("CNN Test Recall: {:.4f}".format(recall))

  probs = model.predict(X_test, verbose=1)
  predicted_classes = model.predict_classes(X_test, verbose=1)


  print(classification_report(y_test, predicted_classes))

  print("Conv Net ends..")
  
convnet_embedding_layer(vocab_size_initial,X_train_initial,X_test_initial,y_train,y_test)


ConvNet Begins...





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1812200   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 128)           38528     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               12900     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,863,729
Trainable params: 1,863,729
Non-trainable params: 0
___________________________________________________________


## LSTM with Embedding learned as part of  model
<br>
<br>
<br>
<br>

In [ ]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report


In [ ]:
def lstm_embedding_layer(vocab_size,X_train,X_test,y_train,y_test):
  early_stopping = [EarlyStopping(monitor='val_loss',
                          min_delta=0,restore_best_weights=True,
                          patience=50,
                          verbose=1, mode='auto')]
  ## Network architecture buraya ekstradann convolutionoal layer eklendi.
  model = keras.Sequential()
  model.add(keras.layers.Embedding(vocab_size, 100, input_length=100))
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Conv1D(128, 3, activation='relu'))
  model.add(keras.layers.MaxPooling1D(pool_size=4))
  model.add(keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2))
  model.add(keras.layers.Dense(100, activation='relu'))
  model.add(keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
  ## Fit the model
  model.fit(X_train, y_train, validation_split=0.4, epochs=150,callbacks=early_stopping)
  loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, y_train, verbose=1)
  print("LSTM Training Loss: {:.4f}".format(loss))
  print("LSTM Training Accuracy: {:.4f}".format(accuracy))
  print("LSTM Training f1 score: {:.4f}".format(f1_score))
  print("LSTM Training Precision: {:.4f}".format(precision))
  print("LSTM Training Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)
  print("LSTM Test Loss: {:.4f}".format(loss))
  print("LSTM Test Accuracy: {:.4f}".format(accuracy))
  print("LSTM Test f1 score: {:.4f}".format(f1_score))
  print("LSTM Test Precision: {:.4f}".format(precision))
  print("LSTM Test Recall: {:.4f}".format(recall))

  probs = model.predict(X_test, verbose=1)
  predicted_classes = model.predict_classes(X_test, verbose=1)


  print(classification_report(y_test, predicted_classes))

  print("LSTM  ends..")
  #print(predicted_classes)
lstm_embedding_layer(vocab_size_initial,X_train_initial,X_test_initial,y_train,y_test)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Train on 7944 samples, validate on 5296 samples
Epoch 1/150
7944/7944 [==============================] - 27s 3ms/step - loss: 0.6420 - acc: 0.6665 - f1_m: 0.0019 - precision_m: 0.0015 - recall_m: 0.0027 - val_loss: 0.6378 - val_acc: 0.6682 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/150
7944/7944 [==============================] - 25s 3ms/step - loss: 0.5577 - acc: 0.7360 - f1_m: 0.4652 - precision_m: 0.6163 - recall_m: 0.4061 - val_loss: 0.5369 - val_acc: 0.7292 - val_f1_m: 0.5923 - val_precision_m: 0.5844 - val_recall_m: 0.6262
Epoch 3/150
7944/7944 [==============================] - 26s 3ms/step - loss: 0.3956 - acc: 0.8420 - f1_m: 0.7362 - precision_m: 0.8050 - recall_m: 0.7007 - val_loss: 0.5399 - val_acc: 0.7460 - val_f1_m: 0.5941 - val_precision_m: 0.6237 - val_recall_m: 0.5892
Epoch 4/150
7944/7944 [====================

## Using Pre-Trained Embedding (Google's trained Word2Vec model)
<br>
<br>
<br>
<br>

In [ ]:
print("Word Embedding Starts... for stemmed tweets")
# train your word embeddings during the training of your neural network. 
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train["tweet_initial_nontoken"])
X_train_initial = tokenizer.texts_to_sequences(X_train["tweet_initial_nontoken"])
X_test_initial = tokenizer.texts_to_sequences(X_test["tweet_initial_nontoken"])
vocab_size_initial = len(tokenizer.word_index) + 1 
wordIndex_initial=tokenizer.word_index # it is my index


tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train["tweet_stopword"])
X_train_stopword=tokenizer.texts_to_sequences(X_train["tweet_stopword"])
X_test_stopword=tokenizer.texts_to_sequences(X_test["tweet_stopword"])
vocab_size_stopword=  len(tokenizer.word_index) + 1 
wordIndex_stopword=tokenizer.word_index # it is my index


tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train["tweet_stemmed"])
X_train_stemmed=tokenizer.texts_to_sequences(X_train["tweet_stemmed"])
X_test_stemmed=tokenizer.texts_to_sequences(X_test["tweet_stemmed"])
vocab_size_stemmed = len(tokenizer.word_index) + 1 
wordIndex_stemmed=tokenizer.word_index # it is my index

y_train=[1 if i == "OFF" else 0 for i in X_train["subtask_a"]]
y_test= [1 if i == "OFF" else 0 for i in labels_a[1]]



print(X_train["tweet_initial_nontoken"][1])
print(X_train_initial[1])
print(X_train["tweet_stopword"][1])
print(X_train_stopword[1])
print(X_train["tweet_stemmed"][1])
print(X_test_stemmed[1])


#implement padding

from keras.preprocessing.sequence import pad_sequences
max_len = 100
X_train_initial = pad_sequences(X_train_initial, padding='post', maxlen=max_len)
X_test_initial = pad_sequences(X_test_initial, padding='post', maxlen=max_len)

X_train_stopword=pad_sequences(X_train_stopword, padding='post', maxlen=max_len)
X_test_stopword=pad_sequences(X_test_stopword, padding='post', maxlen=max_len)

X_train_stemmed=pad_sequences(X_train_stemmed, padding='post', maxlen=max_len)
X_test_stemmed=pad_sequences(X_test_stemmed, padding='post', maxlen=max_len)

print(X_train["tweet_initial_nontoken"][20])
print(X_train_initial[20])

print(X_train["tweet_stopword"][20])
print(X_train_stopword[20])

print(X_train["tweet_stemmed"][20])
print(X_train_stemmed[20])


Word Embedding Starts... for stemmed tweets
user user go home you re drunk user maga url
[1, 1, 90, 310, 5, 196, 949, 1, 42, 17]
user user go home drunk user maga url
[1, 1, 23, 205, 824, 1, 8, 2]
user user go home drunk user maga url
[2260, 8, 61, 22, 131]
user user user user lol throwing the bullshit flag on such nonsense putuporshutup kavanaugh maga callthevotealready url
[   1    1    1    1  155 1449    2  535  811   20  192  975 8643  137
   42 8644   17    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
user user user user lol throwing bullshit flag nonsense putuporshutup kavanaugh maga callthevotealready url
[   1

In [ ]:
  def createEmbeddingLayer(wordIndex,not_static):
    embedding_dim=300
    vocabulary_size=min(len(wordIndex)+1,25000)
    embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
    for word, i in wordIndex.items():
        if i>=25000:
          continue
        try:
            embedding_vector = w2v_model[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
          embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)

    custom_embedding_layer = Embedding(vocabulary_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                trainable=not_static)
    return custom_embedding_layer

  

## Conv Net with word2vec not static.
<br>
<br>
<br>
<br>


In [ ]:
def convnet(vocab_size,X_train,X_test,y_train,y_test,wordIndex):
  print("ConvNet Begins...")
  embedding_dim = 100
  early_stopping = [EarlyStopping(monitor='val_loss',
                          min_delta=0,restore_best_weights=False,
                          patience=50,
                          verbose=1, mode='auto')]
  model = Sequential()
  model.add(createEmbeddingLayer(wordIndex,True))
  model.add(layers.Conv1D(128, 3, activation='relu'))
  model.add(layers.GlobalMaxPooling1D())
  model.add(layers.Dense(100, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['acc',f1_m,precision_m, recall_m])
  model.summary()

  history = model.fit(X_train, y_train,
                      epochs=150,
                      verbose=1,
                      validation_split=0.4,callbacks=early_stopping,
                      batch_size=10)
  loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, y_train, verbose=1)
  print("CNN Training Loss: {:.4f}".format(loss))
  print("CNN Training Accuracy: {:.4f}".format(accuracy))
  print("CNN Training f1 score: {:.4f}".format(f1_score))
  print("CNN Training Precision: {:.4f}".format(precision))
  print("CNN Training Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)
  print("CNN Test Loss: {:.4f}".format(loss))
  print("CNN Test Accuracy: {:.4f}".format(accuracy))
  print("CNN Test f1 score: {:.4f}".format(f1_score))
  print("CNN Test Precision: {:.4f}".format(precision))
  print("CNN Test Recall: {:.4f}".format(recall))

  probs = model.predict(X_test, verbose=1)
  predicted_classes = model.predict_classes(X_test, verbose=1)

  #filename = 'finalized_model_cnn.sav'
  #pickle.dump(model, open(filename, 'wb'))

  print(classification_report(y_test, predicted_classes))

  print("Conv Net ends..")
  
convnet(vocab_size_initial,X_train_initial,X_test_initial,y_train,y_test,wordIndex_initial)


ConvNet Begins...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 300)         5436600   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         115328    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               12900     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 5,564,929
Trainable params: 5,564,929
Non-trainable params: 0
_________________________________________________________________
Train on 7944 samples, validate on 5296 samples
Epoch 1/150
7944/7944 [=====================

## CONV + LSTM Net with word2vec not static
<br>
<br>
<br>
<br>


In [ ]:
  ## Network architecture buraya ekstradann convolutionoal layer eklendi.
  def lstm(vocab_size,X_train,X_test,y_train,y_test,wordIndex):
    early_stopping = [EarlyStopping(monitor='val_loss',
                          min_delta=0,restore_best_weights=True,
                          patience=50,
                          verbose=1, mode='auto')]
    model = Sequential()
    model.add(createEmbeddingLayer(wordIndex,True))
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(layers.Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
    ## Fit the model
    model.fit(X_train, y_train, validation_split=0.4, epochs=150,callbacks=early_stopping)
    loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, y_train, verbose=1)
    print("LSTM Training Loss: {:.4f}".format(loss))
    print("LSTM Training Accuracy: {:.4f}".format(accuracy))
    print("LSTM Training f1 score: {:.4f}".format(f1_score))
    print("LSTM Training Precision: {:.4f}".format(precision))
    print("LSTM Training Recall: {:.4f}".format(recall))

    loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)
    print("LSTM Test Loss: {:.4f}".format(loss))
    print("LSTM Test Accuracy: {:.4f}".format(accuracy))
    print("LSTM Test f1 score: {:.4f}".format(f1_score))
    print("LSTM Test Precision: {:.4f}".format(precision))
    print("LSTM Test Recall: {:.4f}".format(recall))

    probs = model.predict(X_test, verbose=1)
    predicted_classes = model.predict_classes(X_test, verbose=1)


    # = 'finalized_model_lstm.sav'
    #pickle.dump(model, open(filename, 'wb'))

    print(classification_report(y_test, predicted_classes))

    print("LSTM  ends..")

lstm(vocab_size_initial,X_train_initial,X_test_initial,y_train,y_test,wordIndex_initial)



Train on 7944 samples, validate on 5296 samples
Epoch 1/150
7944/7944 [==============================] - 24s 3ms/step - loss: 0.6121 - acc: 0.6814 - f1_m: 0.1453 - precision_m: 0.1960 - recall_m: 0.1341 - val_loss: 0.5413 - val_acc: 0.7273 - val_f1_m: 0.6156 - val_precision_m: 0.5717 - val_recall_m: 0.6900
Epoch 2/150
7944/7944 [==============================] - 21s 3ms/step - loss: 0.4618 - acc: 0.7973 - f1_m: 0.6648 - precision_m: 0.7362 - recall_m: 0.6439 - val_loss: 0.4727 - val_acc: 0.7927 - val_f1_m: 0.6475 - val_precision_m: 0.7161 - val_recall_m: 0.6112
Epoch 3/150
7944/7944 [==============================] - 20s 3ms/step - loss: 0.3397 - acc: 0.8692 - f1_m: 0.7874 - precision_m: 0.8184 - recall_m: 0.7806 - val_loss: 0.5167 - val_acc: 0.7881 - val_f1_m: 0.6236 - val_precision_m: 0.7412 - val_recall_m: 0.5543
Epoch 4/150
7944/7944 [==============================] - 20s 3ms/step - loss: 0.2432 - acc: 0.9119 - f1_m: 0.8617 - precision_m: 0.8713 - recall_m: 0.8684 - val_loss: 0.561

## Conv Net with word2vec static
<br>
<br>
<br>
<br>

In [ ]:
def convnet(vocab_size,X_train,X_test,y_train,y_test,wordIndex):
  print("ConvNet Begins...")
  embedding_dim = 100
  early_stopping = [EarlyStopping(monitor='val_loss',
                          min_delta=0,restore_best_weights=True,
                          patience=50,
                          verbose=1, mode='auto')]
  model = Sequential()
  model.add(createEmbeddingLayer(wordIndex,False))
  model.add(layers.Conv1D(128, 3, activation='relu'))
  model.add(layers.GlobalMaxPooling1D())
  model.add(layers.Dense(100, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['acc',f1_m,precision_m, recall_m])
  model.summary()

  history = model.fit(X_train, y_train,
                      epochs=150,
                      verbose=1,
                      validation_split=0.4,
                      batch_size=10,callbacks=early_stopping)
  loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, y_train, verbose=1)
  print("CNN Training Loss: {:.4f}".format(loss))
  print("CNN Training Accuracy: {:.4f}".format(accuracy))
  print("CNN Training f1 score: {:.4f}".format(f1_score))
  print("CNN Training Precision: {:.4f}".format(precision))
  print("CNN Training Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)
  print("CNN Test Loss: {:.4f}".format(loss))
  print("CNN Test Accuracy: {:.4f}".format(accuracy))
  print("CNN Test f1 score: {:.4f}".format(f1_score))
  print("CNN Test Precision: {:.4f}".format(precision))
  print("CNN Test Recall: {:.4f}".format(recall))

  probs = model.predict(X_test, verbose=1)
  predicted_classes = model.predict_classes(X_test, verbose=1)

  #filename = 'finalized_model_cnn.sav'
  #pickle.dump(model, open(filename, 'wb'))

  print(classification_report(y_test, predicted_classes))

  print("Conv Net ends..")
  
convnet(vocab_size_initial,X_train_initial,X_test_initial,y_train,y_test,wordIndex_initial)



ConvNet Begins...
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 300)         5436600   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         115328    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               12900     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 5,564,929
Trainable params: 128,329
Non-trainable params: 5,436,600
_________________________________________________________________
Train on 7944 samples, validate on 5296 samples
Epoch 1/150
7944/7944 [===============

## CONV + LSTM Net with word2vec static
<br>
<br>
<br>
<br>



In [ ]:
  ## Network architecture buraya ekstradann convolutionoal layer eklendi.
  def conv_lstm(vocab_size,X_train,X_test,y_train,y_test,wordIndex):
    early_stopping = [EarlyStopping(monitor='val_loss',
                          min_delta=0,restore_best_weights=True,
                          patience=50,
                          verbose=1, mode='auto')]
    model = Sequential()
    model.add(createEmbeddingLayer(wordIndex,False))
    model.add(Dropout(0.2))
    model.add(Conv1D(64, 5, activation='relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
, metrics=['acc',f1_m,precision_m, recall_m])
    ## Fit the model
    model.fit(X_train, y_train, validation_split=0.4, epochs=150,callbacks=early_stopping)
    loss, accuracy, f1_score, precision, recall = model.evaluate(X_train, y_train, verbose=1)
    print("LSTM Training Loss: {:.4f}".format(loss))
    print("LSTM Training Accuracy: {:.4f}".format(accuracy))
    print("LSTM Training f1 score: {:.4f}".format(f1_score))
    print("LSTM Training Precision: {:.4f}".format(precision))
    print("LSTM Training Recall: {:.4f}".format(recall))

    loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)
    print("LSTM Test Loss: {:.4f}".format(loss))
    print("LSTM Test Accuracy: {:.4f}".format(accuracy))
    print("LSTM Test f1 score: {:.4f}".format(f1_score))
    print("LSTM Test Precision: {:.4f}".format(precision))
    print("LSTM Test Recall: {:.4f}".format(recall))

    probs = model.predict(X_test, verbose=1)
    predicted_classes = model.predict_classes(X_test, verbose=1)


    # = 'finalized_model_lstm.sav'
    #pickle.dump(model, open(filename, 'wb'))

    print(classification_report(y_test, predicted_classes))

    print("LSTM  ends..")

conv_lstm(vocab_size_initial,X_train_initial,X_test_initial,y_train,y_test,wordIndex_initial)


Train on 7944 samples, validate on 5296 samples
Epoch 1/150
7944/7944 [==============================] - 23s 3ms/step - loss: 0.6065 - acc: 0.6813 - f1_m: 0.2214 - precision_m: 0.3729 - recall_m: 0.1929 - val_loss: 0.5256 - val_acc: 0.7398 - val_f1_m: 0.5805 - val_precision_m: 0.6082 - val_recall_m: 0.5760
Epoch 2/150
7944/7944 [==============================] - 19s 2ms/step - loss: 0.5292 - acc: 0.7486 - f1_m: 0.5683 - precision_m: 0.6695 - recall_m: 0.5394 - val_loss: 0.5084 - val_acc: 0.7481 - val_f1_m: 0.6352 - val_precision_m: 0.5970 - val_recall_m: 0.6996
Epoch 3/150
7944/7944 [==============================] - 20s 3ms/step - loss: 0.4983 - acc: 0.7742 - f1_m: 0.6135 - precision_m: 0.7054 - recall_m: 0.5735 - val_loss: 0.4823 - val_acc: 0.7827 - val_f1_m: 0.6244 - val_precision_m: 0.7097 - val_recall_m: 0.5769
Epoch 4/150
7944/7944 [==============================] - 20s 2ms/step - loss: 0.4708 - acc: 0.7835 - f1_m: 0.6416 - precision_m: 0.6954 - recall_m: 0.6292 - val_loss: 0.494

# PYTORCH 
<br>
<br>
<br>
<br>

## Imports 

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import pandas as pd
import numpy as npC
import torch
import torch.utils.data as data_utils

import torch.nn.functional as F
from torch import nn
from torchsummary import summary
from torch.autograd import Variable
from sklearn import metrics
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

import torch.nn as nn

from torch.nn import ModuleList
# we fix the seeds to get consistent results
SEED = 234
torch.manual_seed(SEED)
np.random.seed(SEED)

#Seed Settings
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.deterministic = True


In [ ]:
#glove
import io
EMBEDDING_FILE = r'/content/drive/My Drive/glove.840B.300d.txt'
embeddings_index = {}
f = open(EMBEDDING_FILE, encoding='utf8')
for line in f:
    values = line.split()
    word = ''.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = np.asarray(coefs , dtype='float32')
f.close()

In [ ]:
embedding_matrix = np.zeros((len(wordIndex_initial) + 1, 300))
for word, i in wordIndex_initial.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


cuda


## Pytorch Metrics
<br>
<br>
<br>
<br>

In [ ]:
def sigmoid(arr):
  x=list(arr)
  r=[]
  for i in x:
    r.append( 1 / (1 + math.exp(-i))) 

  return np.array(r)

def clReport(output,target):
    predict = torch.round(torch.sigmoid(output))
#     print(predict)
    correct = (predict == target).float()
    acc = correct.sum() / len(correct)

    y_true = np.array(target.cpu())

    y_pred = predict.detach().cpu().numpy()

    return classification_report(target.detach().cpu().numpy(), predict.detach().cpu().numpy())

def accuracy2(output, target):
    predict = torch.round(torch.sigmoid(output))
#     print(predict)
    correct = (predict == target).float()
    acc = correct.sum() / len(correct)

    y_true = np.array(target.cpu())
    y_pred = predict.detach().cpu().numpy()

    matrix = metrics.confusion_matrix(y_true, y_pred)
    f1_score = metrics.f1_score(y_true, y_pred, average='macro')

    return acc,f1_score,matrix

def accuracy_train(output, target):
    predict = np.round(sigmoid(output))
#     print(predict)
    correct = (predict == target)
    print(predict.shape)

    #acc = correct.sum() / len(correct)
    acc = metrics.accuracy_score(target,predict)

    #print(acc)
    
    y_true = np.array(target)
    y_pred = predict

    #matrix = metrics.confusion_matrix(y_true, y_pred)
    f1_score = metrics.f1_score(y_true, y_pred, average='macro')

    return acc,f1_score


def accuracy_2(output, target):
    print(output.shape)
    average = 'macro'
    length = len(target)
    if output.shape[-1] != 1:
        output = torch.argmax(output, dim=1).float()

    target = target.float()
    output = output.view(target.shape)
    predict = torch.round(torch.sigmoid(output))
    correct = (predict == target).float()
    acc = correct.sum() / length

    y_true = np.array(target)
    y_pred = predict.detach().numpy()

    matrix = metrics.confusion_matrix(y_true, y_pred)
    f1_score = metrics.f1_score(y_true, y_pred, average=average)
    return acc, f1_score, matrix



## Global Train Function
<br>
<br>
<br>
<br>

In [ ]:
def train(model,epoch,lr,embeddingMatrix,Trainable,train_sent_tensor, train_label_tensor, valid_sent_tensor, valid_label_tensor,device):

  epochs=epoch
  lr=lr
  batch=32
  model = model.to(device)

  model.embedding.weight.data.copy_(torch.from_numpy(embeddingMatrix)) #use own embedding
  model.embedding.weight.require_grad = Trainable
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)

  loss_fn = nn.BCEWithLogitsLoss() # Bu değişecek 3.task'ta


  batchIteration = int(len(train_label_tensor) / batch)
  for epoch in range(1, epochs + 1):

    #Randomness
    np.random.seed(SEED)
    p = np.random.permutation(len(train_label_tensor))
    train_sent_tensor, train_label_tensor = train_sent_tensor[p], train_label_tensor[p]

    ################################################################
    train_sent_tensor=train_sent_tensor.to(device) #This is for cuda
    train_label_tensor=train_label_tensor.to(device) #This is for cuda


    #################################################################
    epoch_loss = []
    trainPrediction=[]
    for i in range(0,batchIteration):
      feature = train_sent_tensor[i * batch:(i+1) * batch]
      target = train_label_tensor[i * batch:(i+1) * batch]


      #print(feature.shape) # 64,100
      #print(target.shape)  # 64,1
#             print(feature)
      ##############################
      model.train().cuda()
      ##############################
            # we zero the gradients as they are not removed automatically
      optimizer.zero_grad()
            # queeze is needed as the predictions are initially size (batch size, 1) and we need to remove the dimension of size 1
      #print(model(feature).squeeze(0))

      #print(model(feature).shape) # return [32,1]
      predictions = model(feature).squeeze(1) #  remove the dimension with length one..
      #print(predictions.shape) #[64]
#            
      #calculate training accuracy

      trainPrediction.append(predictions.cpu().detach().numpy().reshape(1,-1))

      


      loss = loss_fn(predictions, target)



            # calculate the gradient of each parameter
      loss.backward()
            # update the parameters using the gradients and optimizer algorithm
      optimizer.step()
      batch_loss = loss.item() #?
      #print(f'| Epoch: {epoch:02} | Batch: {i: 04} | Train Loss: {batch_loss:.3f}')
      ##############################
      epoch_loss.append(batch_loss)
      ##############################  

    #calculation of train accuracy per epoch


    #####    



    trainPrediction=np.array(trainPrediction).reshape(-1,1)

    realValues=np.array(train_label_tensor.cpu().detach().numpy().reshape(-1,1))[0:trainPrediction.shape[0]]


    #print(trainPrediction[:2])
    #print(trainPrediction.shape)
    #print(realValues.shape)

    acc_train,f1_train=accuracy_train(trainPrediction,realValues)
    #print( train_sent_tensor.shape)
    train_sent_tensor=train_sent_tensor.to(device)
    valid_sent_tensor=valid_sent_tensor.to(device)

    #prediction_train = model(train_sent_tensor)   #hata burada
    prediction_val = model(valid_sent_tensor).squeeze(1)

   # train_acc = accuracy(prediction_train, train_label_tensor)
    valid_acc, valid_f1score, valid_matrix  = accuracy2(prediction_val.to(device), valid_label_tensor.to(device))

    if epoch == epochs:

      print( clReport(prediction_val.to(device), valid_label_tensor.to(device)  ))

#         print(predict_val.shape)
    #print(f'Epoch: {epoch: 03} | Train accuracy: {train_acc * 100: .2f}% | Valid acc: {valid_acc * 100: .2f}%')


    print(f'Epoch: {epoch: 03} Train Loss: {np.mean(epoch_loss): .2f} Train Acc:{acc_train*100: .2f}% Train F1: {f1_train*100: .2f}%  Valid acc: {valid_acc * 100: .2f}% Valid f1:{valid_f1score*100: .2f}%')
    print('Valid confusion matrix: ')
    print(valid_matrix)

# Bidrectional LSTM
<br>
<br>
<br>
<br>

In [ ]:
class LSTM(nn.Module):
  def __init__(self):
    super(LSTM, self).__init__()
    VOCAB_SIZE=18122
    EMBEDDING_DIM=300
    HIDDEN_DIM=8
    OUTPUT_DIM=1
    max_len=25000
    num_classes=1
    self.embedding = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
        # hidden layer
    self.lstm = nn.LSTM(EMBEDDING_DIM,HIDDEN_DIM,1,bidirectional=True)
    self.hidden_dim=HIDDEN_DIM
    self.out = nn.Linear(HIDDEN_DIM*2*2, num_classes)


  def forward(self, x):
    embedded = self.embedding(x)
    states, hidden = self.lstm(embedded.permute([1, 0, 2]))
    encoding = torch.cat([states[0],states[-1]], dim=1)
    out = self.out(encoding)
    return out

In [ ]:
XTrainTens=torch.LongTensor(X_train_initial)
XtestTens=torch.LongTensor(X_test_initial)
ytestTens=torch.FloatTensor( y_test)
y_trainTens=torch.FloatTensor(y_train)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Non Static
lstm_notstatic=LSTM()
train(lstm_notstatic,50,0.0001,embedding_matrix,True, XTrainTens, y_trainTens, XtestTens, ytestTens,device)
#Static
lstm_static=LSTM()

train(lstm_static,50,0.0001,embedding_matrix,False, XTrainTens, y_trainTens, XtestTens, ytestTens,device)

(13216,)
Epoch:  01 Train Loss:  0.65 Train Acc: 64.47% Train F1:  44.39%  Valid acc:  71.98% Valid f1: 43.79%
Valid confusion matrix: 
[[614   6]
 [235   5]]
(13216,)
Epoch:  02 Train Loss:  0.63 Train Acc: 66.71% Train F1:  40.06%  Valid acc:  72.33% Valid f1: 42.78%
Valid confusion matrix: 
[[620   0]
 [238   2]]
(13216,)
Epoch:  03 Train Loss:  0.60 Train Acc: 67.74% Train F1:  43.99%  Valid acc:  73.26% Valid f1: 47.89%
Valid confusion matrix: 
[[615   5]
 [225  15]]
(13216,)
Epoch:  04 Train Loss:  0.54 Train Acc: 73.79% Train F1:  64.80%  Valid acc:  73.37% Valid f1: 52.90%
Valid confusion matrix: 
[[599  21]
 [208  32]]
(13216,)
Epoch:  05 Train Loss:  0.49 Train Acc: 78.94% Train F1:  74.59%  Valid acc:  74.42% Valid f1: 59.68%
Valid confusion matrix: 
[[580  40]
 [180  60]]
(13216,)
Epoch:  06 Train Loss:  0.45 Train Acc: 81.79% Train F1:  78.32%  Valid acc:  75.81% Valid f1: 65.13%
Valid confusion matrix: 
[[564  56]
 [152  88]]
(13216,)
Epoch:  07 Train Loss:  0.41 Train Ac

# CNN with Multiple Filter sizes
<br>
<br>
<br>
<br>

In [ ]:

class CNN(nn.Module):
    """
    DPCNN for sentences classification.
    """
    def __init__(self):
        super(CNN, self).__init__()

        VOCAB_SIZE=18122
        EMBEDDING_DIM=300
        HIDDEN_LAYER=64
        OUTPUT_DIM=1
        MAX_LEN=18122
        OUTPUT_CLASSES=1
        MAX_SENT_LEN=50
        window_size=1

        
        self.EMBEDDING_DIM=EMBEDDING_DIM
        self.MAX_SENT_LEN=MAX_SENT_LEN
        self.embedding = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
      
       # self.embedding.weight = nn.Parameter(embedding)
       # self.embedding.weight.requires_grad = update_embedding
        self.dropout = nn.Dropout(p=0.1) #  dropout layer

        #KERNEL_SIZE=3

        KERNEL_SIZES = [3,1]
        self.NUM_KERNEL = 32
        self.KERNEL_COUNT=len(KERNEL_SIZES)
        #self.conv_region_embedding = nn.Conv2d(1, self.num_filter, (filter_size, EMBEDDING_DIM), stride=2)
        #self.conv = nn.Conv1d(in_channels=1, out_channels=64,  kernel_size=(KERNEL_SIZE, EMBEDDING_DIM), stride=1) #conv for 1 kernel size
        
        self.convolution=nn.ModuleList([nn.Conv2d(1,32, (K, EMBEDDING_DIM)) for K in KERNEL_SIZES])
        
        #self.maxpool = nn.MaxPool1d(kernel_size=5)
        self.fully_connected = nn.Linear(self.NUM_KERNEL*self.KERNEL_COUNT, OUTPUT_CLASSES)
        self.rel = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.soft=nn.Softmax()

    def forward(self, x, lengths=None):
        x = self.embedding(x)
        
        x = x.unsqueeze(1)
        x = self.dropout(x)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convolution] 
        #x = self.conv(x) # x becomes feature map
        #x=x.squeeze(3)
        #x= self.rel(x)

        

        #pooled = F.max_pool1d(feature_maps, feature_maps.shape[2])
        x = [F.max_pool1d(i, i.shape[2]).squeeze(2) for i in x]  
        
        #print('IAMHERE2')
      # x = x.squeeze(2)
        
        x = torch.cat(x, 1)
        
        x = self.dropout(x)

        #print('here')
        output = self.sig(self.fully_connected(x))

        return output

In [ ]:
XTrainTens=torch.LongTensor(X_train_initial)
XtestTens=torch.LongTensor(X_test_initial)
ytestTens=torch.FloatTensor( y_test)
y_trainTens=torch.FloatTensor(y_train)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cnn_notstatic=CNN()
#Non Static
train(cnn_notstatic,50,0.001,embedding_matrix,True,XTrainTens, y_trainTens, XtestTens, ytestTens,device)
#Static
cnn_static=CNN()

train(cnn_static,50,0.001,embedding_matrix,False,XTrainTens, y_trainTens, XtestTens, ytestTens,device)


(13216,)
Epoch:  01 Train Loss:  0.70 Train Acc: 33.24% Train F1:  24.95%  Valid acc:  27.91% Valid f1: 21.82%
Valid confusion matrix: 
[[  0 620]
 [  0 240]]
(13216,)
Epoch:  02 Train Loss:  0.69 Train Acc: 33.26% Train F1:  24.96%  Valid acc:  27.79% Valid f1: 21.75%
Valid confusion matrix: 
[[  0 620]
 [  1 239]]
(13216,)
Epoch:  03 Train Loss:  0.69 Train Acc: 33.24% Train F1:  24.95%  Valid acc:  28.60% Valid f1: 22.90%
Valid confusion matrix: 
[[  6 614]
 [  0 240]]
(13216,)
Epoch:  04 Train Loss:  0.69 Train Acc: 33.24% Train F1:  24.95%  Valid acc:  29.42% Valid f1: 24.33%
Valid confusion matrix: 
[[ 15 605]
 [  2 238]]
(13216,)
Epoch:  05 Train Loss:  0.69 Train Acc: 33.22% Train F1:  24.93%  Valid acc:  30.23% Valid f1: 25.90%
Valid confusion matrix: 
[[ 26 594]
 [  6 234]]
(13216,)
Epoch:  06 Train Loss:  0.69 Train Acc: 33.25% Train F1:  24.95%  Valid acc:  27.91% Valid f1: 21.82%
Valid confusion matrix: 
[[  0 620]
 [  0 240]]
(13216,)
Epoch:  07 Train Loss:  0.66 Train Ac

# Deep Cnn
<br>
<br>
<br>
<br>

In [ ]:
class DPCNN(nn.Module):
    """
    DPCNN for sentences classification.
    """
    def __init__(self):
        super(DPCNN, self).__init__()

        VOCAB_SIZE=18122
        EMBEDDING_DIM=300
        HIDDEN_LAYER=64
        OUTPUT_DIM=1
        MAX_LEN=18122
        OUTPUT_CLASSES=1
        MAX_SENT_LEN=50
        window_size=1

        
        self.EMBEDDING_DIM=EMBEDDING_DIM
        self.MAX_SENT_LEN=MAX_SENT_LEN
        self.embedding = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
      
       # self.embedding.weight = nn.Parameter(embedding)
       # self.embedding.weight.requires_grad = update_embedding
        self.dropout = nn.Dropout(p=0.1) #  dropout layer


        KERNEL_SIZES = [1,3]
        self.NUM_KERNEL = 32
        self.KERNEL_COUNT=len(KERNEL_SIZES)
        #self.conv_region_embedding = nn.Conv2d(1, self.num_filter, (filter_size, EMBEDDING_DIM), stride=2)
        self.conv1 = nn.Conv1d(in_channels=300, out_channels=64,  kernel_size=(KERNEL_SIZES[1]), stride=1 ) #conv for 1 kernel size
        

        self.conv2 = nn.Conv1d(in_channels=64, out_channels=32,  kernel_size=(KERNEL_SIZES[0]), stride=1 ) #conv for 1 kernel size

       
        

        #self.maxpool = nn.MaxPool1d(kernel_size=5)
        self.fully_connected = nn.Linear(32, OUTPUT_CLASSES)
        self.rel = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.soft=nn.Softmax()

    def forward(self, x, lengths=None):
        x = self.embedding(x)
        #print(x.shape)
        #x = x.unsqueeze(1)
        x = self.dropout(x)
        x = x.permute(0, 2, 1)
        

        #x=x.squeeze(1) # After embedding, i must remove 1st dimension every time ! 
        #Now x becomes 32,50,300
        #print(x.shape) # torch.Size([32, 1, 50, 300])
        x = self.rel(self.conv1(x))
        #print('conv_end')
        #print(x.shape)  # ([32, 32, 48, 1]) if kernel size =3
                         # ([32, 32, 50, 1])  if kernel size = 1 no padding.


        #x=x.squeeze(3)
        
        # x becomes ([32, 32, 50])  if kernel size = 1 no padding.
        x = F.max_pool1d(x, x.shape[2]) 

        # x becomes ([32, 32, 1])  
        #print('pooling end')
        #print(x.shape)

        
        x = self.rel(self.conv2(x))
        #print('here')
#        x = self.rel(self.conv2(x)).squeeze(3) 

        x = F.max_pool1d(x, x.shape[2]) 

        #x = x.squeeze(2)


        x# = F.max_pool1d(x, x.shape[2]) 

        #x = x.squeeze(2)

        x = self.dropout(x)
        x=x.squeeze(2)
        output = self.sig(self.fully_connected(x))

        return output

In [ ]:
XTrainTens=torch.LongTensor(X_train_initial)
XtestTens=torch.LongTensor(X_test_initial)
ytestTens=torch.FloatTensor( y_test)
y_trainTens=torch.FloatTensor(y_train)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



deepCNN_notstatic=DPCNN()
#Non Static
train(deepCNN_notstatic,100,0.001,embedding_matrix,True,XTrainTens, y_trainTens, XtestTens, ytestTens,device)
#Static
deepCNN_static=DPCNN()

train(deepCNN_static,100,0.001,embedding_matrix,False,XTrainTens, y_trainTens, XtestTens, ytestTens,device)


(13216,)
Epoch:  01 Train Loss:  0.70 Train Acc: 33.24% Train F1:  24.95%  Valid acc:  27.79% Valid f1: 22.18%
Valid confusion matrix: 
[[  4 616]
 [  5 235]]
(13216,)
Epoch:  02 Train Loss:  0.69 Train Acc: 33.26% Train F1:  24.96%  Valid acc:  29.77% Valid f1: 26.24%
Valid confusion matrix: 
[[ 34 586]
 [ 18 222]]
(13216,)
Epoch:  03 Train Loss:  0.69 Train Acc: 33.24% Train F1:  24.95%  Valid acc:  33.84% Valid f1: 31.62%
Valid confusion matrix: 
[[ 68 552]
 [ 17 223]]
(13216,)
Epoch:  04 Train Loss:  0.69 Train Acc: 33.24% Train F1:  24.95%  Valid acc:  33.37% Valid f1: 31.07%
Valid confusion matrix: 
[[ 65 555]
 [ 18 222]]
(13216,)
Epoch:  05 Train Loss:  0.69 Train Acc: 33.22% Train F1:  24.93%  Valid acc:  32.67% Valid f1: 29.77%
Valid confusion matrix: 
[[ 53 567]
 [ 12 228]]
(13216,)
Epoch:  06 Train Loss:  0.69 Train Acc: 33.25% Train F1:  24.95%  Valid acc:  28.14% Valid f1: 22.18%
Valid confusion matrix: 
[[  2 618]
 [  0 240]]
(13216,)
Epoch:  07 Train Loss:  0.66 Train Ac

# GRU
<br>
<br>
<br>
<br>

In [ ]:
class GRU(nn.Module):
  def __init__(self):
    super(GRU, self).__init__()
    epochs=10
    VOCAB_SIZE=18122
    EMBEDDING_DIM=300
    HIDDEN_DIM=8
    OUTPUT_DIM=1
    max_len=25000
    num_classes=1
    self.embedding = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
        # hidden layer
    self.GRU = nn.GRU(EMBEDDING_DIM,HIDDEN_DIM,1,bidirectional=True)
    self.hidden_dim=HIDDEN_DIM
    self.out = nn.Linear(HIDDEN_DIM*2*2, num_classes)


  def forward(self, x):
    embedded = self.embedding(x)
    states, hidden = self.GRU(embedded.permute([1, 0, 2]))
    encoding = torch.cat([states[0],states[-1]], dim=1)
    out = self.out(encoding)
    return out


In [ ]:
XTrainTens=torch.LongTensor(X_train_initial)
XtestTens=torch.LongTensor(X_test_initial)
ytestTens=torch.FloatTensor( y_test)
y_trainTens=torch.FloatTensor(y_train)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Not Static
gru_notstatic=GRU()

train(gru_notstatic,50,0.0001,embedding_matrix,True, XTrainTens, y_trainTens, XtestTens, ytestTens,device)
#Static
gru_static=GRU()

train(gru_static,50,0.0001,embedding_matrix,False, XTrainTens, y_trainTens, XtestTens, ytestTens,device)

(13216,)
Epoch:  01 Train Loss:  0.64 Train Acc: 65.94% Train F1:  40.57%  Valid acc:  70.70% Valid f1: 43.97%
Valid confusion matrix: 
[[601  19]
 [233   7]]
(13216,)
Epoch:  02 Train Loss:  0.64 Train Acc: 66.63% Train F1:  40.20%  Valid acc:  71.86% Valid f1: 42.99%
Valid confusion matrix: 
[[615   5]
 [237   3]]
(13216,)
Epoch:  03 Train Loss:  0.62 Train Acc: 66.73% Train F1:  40.18%  Valid acc:  71.98% Valid f1: 42.25%
Valid confusion matrix: 
[[618   2]
 [239   1]]
(13216,)
Epoch:  04 Train Loss:  0.59 Train Acc: 67.90% Train F1:  44.20%  Valid acc:  73.37% Valid f1: 46.94%
Valid confusion matrix: 
[[619   1]
 [228  12]]
(13216,)
Epoch:  05 Train Loss:  0.53 Train Acc: 74.36% Train F1:  62.81%  Valid acc:  76.16% Valid f1: 57.83%
Valid confusion matrix: 
[[611   9]
 [196  44]]
(13216,)
Epoch:  06 Train Loss:  0.47 Train Acc: 80.37% Train F1:  75.35%  Valid acc:  79.77% Valid f1: 69.05%
Valid confusion matrix: 
[[596  24]
 [150  90]]
(13216,)
Epoch:  07 Train Loss:  0.41 Train Ac

# LSTM + CNN
<br>
<br>
<br>
<br>

In [ ]:
class LSTMCNN(nn.Module):
    """
    DPCNN for sentences classification.
    """
    def __init__(self):
        super(LSTMCNN, self).__init__()

        VOCAB_SIZE=18122
        EMBEDDING_DIM=300
        HIDDEN_LAYER=64
        OUTPUT_DIM=1
        MAX_LEN=18122
        OUTPUT_CLASSES=1
        MAX_SENT_LEN=50
        window_size=1
        HIDDEN_DIM=8

        
        self.EMBEDDING_DIM=EMBEDDING_DIM
        self.MAX_SENT_LEN=MAX_SENT_LEN
        self.embedding = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
      
       # self.embedding.weight = nn.Parameter(embedding)
       # self.embedding.weight.requires_grad = update_embedding
        self.dropout = nn.Dropout(p=0.1) #  dropout layer


        KERNEL_SIZES = [3,1]
        self.NUM_KERNEL = 32
        self.KERNEL_COUNT=len(KERNEL_SIZES)
        #self.conv_region_embedding = nn.Conv2d(1, self.num_filter, (filter_size, EMBEDDING_DIM), stride=2)
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32,  kernel_size=(KERNEL_SIZES[0], EMBEDDING_DIM), stride=1 ) #conv for 1 kernel size
        self.lstm = nn.LSTM(EMBEDDING_DIM,HIDDEN_DIM,1,bidirectional=True)


        #self.conv2 = nn.Conv1d(in_channels=32, out_channels=32,  kernel_size=(KERNEL_SIZES[1], 1), stride=1 ) #conv for 1 kernel size

       
        

        #self.maxpool = nn.MaxPool1d(kernel_size=5)
        self.fully_connected = nn.Linear(HIDDEN_DIM*2*2, OUTPUT_CLASSES)
        self.rel = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.soft=nn.Softmax()

    def forward(self, x, lengths=None):
        x = self.embedding(x)
        states, hidden = self.lstm(x)
        encoding = torch.cat([states[0],states[-1]], dim=1)
        x = x.unsqueeze(1)
        x = self.dropout(x)

        x = self.rel(self.conv1(x))
        x = x.squeeze(3) 
        x = F.max_pool1d(x, x.shape[2]) 

        x = x.squeeze(2)

        x = self.dropout(x)

      

        output = self.sig(self.fully_connected(x))

        return output

In [ ]:

XTrainTens=torch.LongTensor(X_train_initial)
XtestTens=torch.LongTensor(X_test_initial)
ytestTens=torch.FloatTensor( y_test)
y_trainTens=torch.FloatTensor(y_train)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

LSTMCNN_notstatic=LSTMCNN()

#Not Static
train(LSTMCNN_notstatic,50,0.001,embedding_matrix,True, XTrainTens, y_trainTens, XtestTens, ytestTens,device)
#Static
LSTMCNN_static=LSTMCNN()

train(LSTMCNN_static,50,0.001,embedding_matrix,False, XTrainTens, y_trainTens, XtestTens, ytestTens,device)


(13216,)
Epoch:  01 Train Loss:  0.70 Train Acc: 33.24% Train F1:  24.95%  Valid acc:  27.91% Valid f1: 21.82%
Valid confusion matrix: 
[[  0 620]
 [  0 240]]
(13216,)
Epoch:  02 Train Loss:  0.69 Train Acc: 33.26% Train F1:  24.96%  Valid acc:  27.91% Valid f1: 21.82%
Valid confusion matrix: 
[[  0 620]
 [  0 240]]
(13216,)
Epoch:  03 Train Loss:  0.69 Train Acc: 33.24% Train F1:  24.95%  Valid acc:  28.14% Valid f1: 22.39%
Valid confusion matrix: 
[[  4 616]
 [  2 238]]
(13216,)
Epoch:  04 Train Loss:  0.69 Train Acc: 33.24% Train F1:  24.95%  Valid acc:  29.65% Valid f1: 24.58%
Valid confusion matrix: 
[[ 16 604]
 [  1 239]]
(13216,)
Epoch:  05 Train Loss:  0.69 Train Acc: 33.22% Train F1:  24.93%  Valid acc:  28.84% Valid f1: 24.05%
Valid confusion matrix: 
[[ 16 604]
 [  8 232]]
(13216,)
Epoch:  06 Train Loss:  0.69 Train Acc: 33.25% Train F1:  24.95%  Valid acc:  27.91% Valid f1: 21.82%
Valid confusion matrix: 
[[  0 620]
 [  0 240]]
(13216,)
Epoch:  07 Train Loss:  0.66 Train Ac

# Keras Attention
<br>
<br>
<br>
<br>

In [ ]:
from keras import Input
from keras.layers import Concatenate

def createEmbeddingLayer(wordIndex,not_static):
    embedding_dim=300
    vocabulary_size=min(len(wordIndex)+1,25000)
    embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
    for word, i in wordIndex.items():
        if i>=25000:
          continue
        try:
            embedding_vector = w2v_model[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
          embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)

    custom_embedding_layer = Embedding(vocabulary_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                trainable=not_static)
    return custom_embedding_layer

In [ ]:
from keras import backend as K
from keras import initializers, regularizers, constraints
from keras.engine.topology import Layer

class Attention_feed_forward(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
     
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention_feed_forward, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
  
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

In [ ]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Dropout, Bidirectional, CuDNNLSTM



class Attention_BiLSTM(object):
    def __init__(self, maxlen, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))
        embedding = createEmbeddingLayer(wordIndex_initial,False)(input)
        x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(embedding)  # LSTM or GRU
        x = Attention_feed_forward(self.maxlen)(x)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

#Benefited from https://github.com/ShawnyXiao/TextClassification-Keras#5-textattbirnn


In [ ]:
 def sigmoid(x): 
    return 1.0/(1 + np.exp(-x))


In [ ]:
def train_att(X_train_initial,X_test_initial,y_train,y_test) : 
  bilstm_attention_model= Attention_BiLSTM(100, 300).get_model()

  early_stopping = [EarlyStopping(monitor='val_acc',min_delta=0,restore_best_weights=True, patience=50,verbose=1, mode='auto')]
  bilstm_attention_model.compile(optimizer=keras.optimizers.Adam(lr=0.0005),
                  loss='binary_crossentropy',
                  metrics=['acc',f1_m,precision_m, recall_m])
  bilstm_attention_model.summary()

  bilstm_attention_model.fit(X_train_initial, y_train,
            batch_size=32,
            epochs=150,
            callbacks=early_stopping,
            validation_split=0.1)

  loss, accuracy, f1_score, precision, recall = bilstm_attention_model.evaluate(X_train_initial, y_train, verbose=1)
  print("Attention- BiRNN Training Loss: {:.4f}".format(loss))
  print("Attention- BiRNN Accuracy: {:.4f}".format(accuracy))
  print("Attention- BiRNN f1 score: {:.4f}".format(f1_score))
  print("Attention- BiRNN Precision: {:.4f}".format(precision))
  print("Attention- BiRNN Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = bilstm_attention_model.evaluate(X_test_initial, y_test, verbose=1)
  print("Attention- BiRNN Test Loss: {:.4f}".format(loss))
  print("Attention- BiRNN Test Accuracy: {:.4f}".format(accuracy))
  print("Attention- BiRNN Test f1 score: {:.4f}".format(f1_score))
  print("Attention- BiRNN Test Precision: {:.4f}".format(precision))
  print("Attention- BiRNN Test Recall: {:.4f}".format(recall))
 
  probs = bilstm_attention_model.predict(X_test_initial, verbose=1)
  print('lenght of probs : ' ,len(probs))
  #predicted_classes = probs.argmax(axis=-1)
  predicted_classes=[0 if i < 0.5 else 1 for i in probs]
  #predicted_classes = bilstm_attention_model.predict_classes(X_test_initial, verbose=1)
  #print(predicted_classes)

  print(classification_report(y_test, predicted_classes))

train_att(np.array(X_train_initial),np.array(X_test_initial),np.array(y_train),np.array(y_test))



Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 100, 300)          5436600   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 256)          440320    
_________________________________________________________________
attention_2 (Attention)      (None, 256)               356       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 257       
Total params: 5,877,533
Trainable params: 440,933
Non-trainable params: 5,436,600
_________________________________________________________________
Train on 11916 samples, validate on 1324 samples
Epoch 1/150
11916/11916 [==============================] - 2

In [ ]:
probs

NameError: ignored

#BERT

<br>
<br>
<br>

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report
def recall_m(true_Y, pred_Y):
        TP = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        possible_pos = K.sum(K.round(K.clip(true_Y, 0, 1)))
        rec = TP / (possible_pos + K.epsilon())
        return rec

def precision_m(true_Y, pred_Y):
        true_positives = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(pred_Y, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(true_Y, pred_Y):
    pres = precision_m(true_Y, pred_Y)
    rec = recall_m(true_Y, pred_Y)
    return 2*((pres*rec)/(pres+rec+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
!pip install pytorch_pretrained_bert
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
#Data pre-processing before BERT
tweets_train = X_train.tweet_initial_nontoken.values
tweets_test = X_test.tweet_initial_nontoken.values


tweets_train = ["[CLS] " + tweet_initial_nontoken + " [SEP]" for tweet_initial_nontoken in tweets_train]
tweets_test = ["[CLS] " + tweet_initial_nontoken + " [SEP]" for tweet_initial_nontoken in tweets_test]

In [ ]:
X=tweets_train+tweets_test
Y=y_train+y_test

In [ ]:
print(np.array(X).shape)
print(np.array(Y).shape)

print(X_train.shape)
print(X_test.shape)

(14100,)
(14100,)
(13240, 9)
(860, 8)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_X = [tokenizer.tokenize(sent) for sent in X]


print ("Tokenize the first sentence:")
print (tokenized_X[0])


Tokenize the first sentence:
['[CLS]', 'user', 'she', 'should', 'ask', 'a', 'few', 'native', 'americans', 'what', 'their', 'take', 'on', 'this', 'is', '[SEP]']


In [ ]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_X]

# Pad our input tokens
MAX_LEN=120
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)


# Use train_test_split to split our data into train and validation sets for training
#train_inputs = input_ids[0:13240]
#validation_inputs=input_ds[13240:]
#train_labels=Y[0:13240]
#validation_labels=Y[13240:]
#train_masks=[atten]


#küçük bir hesaplama gerekiyor!! test_size = 860/14100
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, Y, 
                                                            random_state=2018, test_size=860/14100)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=860/14100)

print(train_inputs.shape)
print(validation_inputs.shape)
print(train_labels)
print(validation_labels)


(13240, 120)
(860, 120)
[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 

In [ ]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [ ]:

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]



In [ ]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     )

t_total value of -1 results in schedule not being applied


In [ ]:
# Function to calculate the accuracy of our predictions vs labels
from sklearn.metrics import confusion_matrix
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    report=classification_report(labels_flat , pred_flat)
    
    print('pred_flat:',pred_flat)
    print('labels:',labels_flat)
    


    return (np.sum(pred_flat == labels_flat) / len(labels_flat)) ,report
def concat(preds,labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    report=classification_report(labels_flat , pred_flat)
    
    print('pred_flat:',pred_flat)
    print('labels:',labels_flat)


    return list(labels_flat) ,list(pred_flat)

# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 8

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0

  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass

    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels) # CUDA OUT OF MEMORY

    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()

    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  LABELS=[]
  PREDICTION=[]
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy,report = flat_accuracy(logits, label_ids)
    p,l=concat(logits, label_ids)
    LABELS+=l
    PREDICTION+=p    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
    
  report=classification_report(LABELS , PREDICTION)  
  print(report)
  print('confusion_matrix :', confusion_matrix(LABELS, PREDICTION))

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Train loss: 0.4712742041202559
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 1]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 1]
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 1]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 1]
pred_flat: [1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1]
labels: [1 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1]
pred_flat: [1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1]
labels: [1 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1]
pred_flat: [0 0 0 0 1 0 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0]
labels: [0 1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0]
pred_flat: [0 0 0 0 1 0 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0]
labels: [0 1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0]
pred_flat: [1 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0

Epoch:  12%|█▎        | 1/8 [09:48<1:08:42, 588.93s/it]

pred_flat: [0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
pred_flat: [0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
              precision    recall  f1-score   support

           0       0.89      0.83      0.86       616
           1       0.64      0.75      0.69       244

    accuracy                           0.81       860
   macro avg       0.77      0.79      0.78       860
weighted avg       0.82      0.81      0.81       860

confusion_matrix : [[514 102]
 [ 61 183]]
Validation Accuracy: 0.8111772486772487
Train loss: 0.3340197319641781
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 1]
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0

Epoch:  25%|██▌       | 2/8 [19:37<58:53, 588.90s/it]  

...

pred_flat: [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
pred_flat: [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       574
           1       0.67      0.66      0.67       286

    accuracy                           0.78       860
   macro avg       0.75      0.75      0.75       860
weighted avg       0.78      0.78      0.78       860

confusion_matrix : [[479  95]
 [ 96 190]]
Validation Accuracy: 0.7786044973544973
Train loss: 0.0353348590163649
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 1]
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 1 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0

Epoch:  88%|████████▊ | 7/8 [1:08:47<09:49, 589.69s/it]

pred_flat: [0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
pred_flat: [0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
              precision    recall  f1-score   support

           0       0.85      0.84      0.84       582
           1       0.67      0.69      0.68       278

    accuracy                           0.79       860
   macro avg       0.76      0.76      0.76       860
weighted avg       0.79      0.79      0.79       860

confusion_matrix : [[488  94]
 [ 87 191]]
Validation Accuracy: 0.7898478835978836
Train loss: 0.02760725040697371
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 1]
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 

Epoch: 100%|██████████| 8/8 [1:18:36<00:00, 589.46s/it]

pred_flat: [0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
pred_flat: [0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       590
           1       0.66      0.70      0.68       270

    accuracy                           0.79       860
   macro avg       0.76      0.77      0.76       860
weighted avg       0.80      0.79      0.80       860

confusion_matrix : [[494  96]
 [ 81 189]]
Validation Accuracy: 0.7944775132275133


#XLNET

<br>
<br>
<br>

In [ ]:
!pip install pytorch-transformers
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from pytorch_transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
#Data pre-processing before BERT
tweets_train = X_train.tweet_initial_nontoken.values
tweets_test = X_test.tweet_initial_nontoken.values


tweets_train = [tweet_initial_nontoken + " [SEP] [CLS]" for tweet_initial_nontoken in tweets_train]
tweets_test = [ tweet_initial_nontoken + " [SEP] [CLS]" for tweet_initial_nontoken in tweets_test]

X=tweets_train+tweets_test
Y=y_train+y_test

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

tokenized_X = [tokenizer.tokenize(sent) for sent in X]
print ("Tokenize the first sentence:")
print (tokenized_X[0])

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_X]

# Pad our input tokens
MAX_LEN=120
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

#küçük bir hesaplama gerekiyor!! test_size = 860/14100
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, Y, 
                                                            random_state=2018, test_size=860/14100)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=860/14100)

print(train_inputs.shape)
print(validation_inputs.shape)
print(train_labels)
print(validation_labels)

# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)



Tokenize the first sentence:
['▁user', '▁she', '▁should', '▁ask', '▁a', '▁few', '▁native', '▁', 'american', 's', '▁what', '▁their', '▁take', '▁on', '▁this', '▁is', '▁[', 's', 'ep', ']', '▁[', 'cl', 's', ']']
(13240, 120)
(860, 120)
[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [ ]:

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5,
                     )

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
from sklearn.metrics import confusion_matrix
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    report=classification_report(labels_flat , pred_flat)
    
    print('pred_flat:',pred_flat)
    print('labels:',labels_flat)
    


    return (np.sum(pred_flat == labels_flat) / len(labels_flat)) ,report
def concat(preds,labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    report=classification_report(labels_flat , pred_flat)
    
    print('pred_flat:',pred_flat)
    print('labels:',labels_flat)


    return list(labels_flat) ,list(pred_flat)

# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 8
# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  LABELS=[]
  PREDICTION=[]
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      logits = output[0]
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy,report = flat_accuracy(logits, label_ids)    
    p,l=concat(logits, label_ids)
    LABELS+=l
    PREDICTION+=p   
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
    
  report=classification_report(LABELS , PREDICTION)  
  print(report)
  print('confusion_matrix :', confusion_matrix(LABELS, PREDICTION))
  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Train loss: 0.5215217381859747
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 0 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 1]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 1]
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 0 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 1]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 1]
pred_flat: [1 1 0 1 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1]
labels: [1 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1]
pred_flat: [1 1 0 1 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1]
labels: [1 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1]
pred_flat: [0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0]
labels: [0 1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0]
pred_flat: [0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0]
labels: [0 1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 0]
pred_flat: [0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 0 1

Epoch:  12%|█▎        | 1/8 [13:00<1:31:01, 780.27s/it]

pred_flat: [0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
pred_flat: [0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       585
           1       0.66      0.68      0.67       275

    accuracy                           0.79       860
   macro avg       0.75      0.76      0.76       860
weighted avg       0.79      0.79      0.79       860

confusion_matrix : [[488  97]
 [ 87 188]]
Validation Accuracy: 0.7867063492063492
Train loss: 0.41604933175055875
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 1]
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 

Epoch:  25%|██▌       | 2/8 [26:00<1:18:01, 780.26s/it]

...

pred_flat: [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
pred_flat: [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
              precision    recall  f1-score   support

           0       0.81      0.82      0.82       567
           1       0.65      0.63      0.64       293

    accuracy                           0.76       860
   macro avg       0.73      0.73      0.73       860
weighted avg       0.76      0.76      0.76       860

confusion_matrix : [[467 100]
 [108 185]]
Validation Accuracy: 0.7589285714285714
Train loss: 0.12361355615860742
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 1]
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 

Epoch:  88%|████████▊ | 7/8 [1:31:00<13:00, 780.09s/it]

pred_flat: [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
pred_flat: [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
              precision    recall  f1-score   support

           0       0.81      0.83      0.82       558
           1       0.67      0.64      0.65       302

    accuracy                           0.76       860
   macro avg       0.74      0.73      0.74       860
weighted avg       0.76      0.76      0.76       860

confusion_matrix : [[465  93]
 [110 192]]
Validation Accuracy: 0.7647156084656084
Train loss: 0.09313824945376907
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 1]
pred_flat: [0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0]
labels: [0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 

Epoch: 100%|██████████| 8/8 [1:44:00<00:00, 780.10s/it]

pred_flat: [0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
pred_flat: [0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
labels: [0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1]
              precision    recall  f1-score   support

           0       0.77      0.82      0.79       542
           1       0.66      0.59      0.62       318

    accuracy                           0.73       860
   macro avg       0.71      0.70      0.71       860
weighted avg       0.73      0.73      0.73       860

confusion_matrix : [[444  98]
 [131 187]]
Validation Accuracy: 0.734457671957672


In [ ]:
X_train

NameError: ignored

#ELMO
<br>
<br>
<br>

In [ ]:
# Elmo

import tensorflow as tf
import tensorflow_hub as hub

url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

from keras.layers import Input, Lambda, Dense,Layer
from keras.models import Model
from keras import backend as K

# Initialize session
sess = tf.Session()
K.set_session(sess)

In [ ]:
trainx=X_train['tweet_initial_nontoken'].tolist()
trainx = [' '.join(t.split()[0:150]) for t in trainx]
trainx = np.array(trainx, dtype=object)[:, np.newaxis]

testx = X_test['tweet_initial_nontoken'].tolist()
testx = [' '.join(t.split()[0:150]) for t in testx]
testx = np.array(testx, dtype=object)[:, np.newaxis]

In [ ]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

      #  self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
      #  super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [ ]:

# Function to build model
"""
  model.add(Conv1D(128, 3, activation='relu'))
  model.add(MaxPooling1D(pool_size=4))
  model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
  model.add(layers.Dense(100, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))"""

def build_model(): 
  input_text = layers.Input(shape=(1,), dtype="string")
  embedding = ElmoEmbeddingLayer()(input_text)
  #(embedding.shape)
  #conv#1 = Conv1D(128, kernel_size=3, activation='relu')(K.expand_dims(embedding,0))
  #pool1 = MaxPooling1D(pool_size=4)(conv1)
  #conv2 = Conv1D(16, kernel_size=1, activation='relu')(pool1)
  #pol2 = MaxPooling1D(pool_size=2)(conv2)
  #flat = Flatten()(K.get_shape(pool1,-1,1))
  #print("a")
  dense = layers.Dense(128, activation='relu')(embedding)

  pred = layers.Dense(1, activation='sigmoid')(dense)

  model = Model(inputs=[input_text], outputs=pred)

  #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  
  return model

In [ ]:

model_elmo=build_model()
early_stopping = [EarlyStopping(monitor='val_acc',min_delta=0,restore_best_weights=True, patience=3,verbose=1, mode='auto')]
model_elmo.compile(optimizer=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False),loss='binary_crossentropy',metrics=['acc',f1_m,precision_m, recall_m])

model_elmo.fit(trainx, y_train,
            batch_size=16,
            epochs=5,
            callbacks=early_stopping,
            validation_split=0.1)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
elmo_embedding_layer_1 (Elmo (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 131,329
Trainable params: 131,329
Non-trainable params: 0
_________________________________________________________________



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 11916 samples, validate on 1324 samples
Epoch 1/5



11916/11916 [==============================] - 198s 17ms/step - loss: 0.5668 - acc: 0.7062 - f1_m: 0.3057 - precision_m: 0.4938 - recall_m: 0.2513 - val_loss: 0.5406 - val_acc: 0.7107 - val_f1_m: 0.2914 - val_precision_m: 0.5571 - val_recall_m: 0.2134
Epoch 2/5
11916/11916 [==============================] - 177s 15ms/step - loss: 0.5152 - acc: 0.7450 - f1_m: 0.4974 - precision_m: 0.6483 - recall_m: 0.4497 - val_loss: 0.5135 - val_acc: 0.7417 - val_f1_m: 0.5499 - val_precision_m: 0.6346 - val_recall_m: 0.5238
Epoch 3/5
11916/11916 [==============================] - 177s 15ms/step - loss: 0.5010 - acc: 0.7542 - f1_m: 0.5401 - precision_m: 0.6760 - recall_m: 0.4934 - val_loss: 0.5140 - val_acc: 0.7432 - val_f1_m: 0.4621 - val_precision_m: 0.6434 - val_recall_m: 0.3833
Epoch 4/5
11916/11916 [==============================] - 174s 15ms/step - loss: 0.4948 - acc: 0.7638 - f1_m: 0.5621 - precision_m: 0.6897 - recall_m: 0.5213 - val_loss: 0.5005 - val_acc: 0.7485 - val_f1_m: 0.5331 - val_preci

In [ ]:
loss, accuracy, f1_score, precision, recall = model_elmo.evaluate(testx, y_test, verbose=1)
print("elmo Test Loss: {:.4f}".format(loss))
print("elmo Test Accuracy: {:.4f}".format(accuracy))
print("elmo Test f1 score: {:.4f}".format(f1_score))
print("elmo  Test Precision: {:.4f}".format(precision))
print("elmo BiRNN Test Recall: {:.4f}".format(recall))
model_elmo.save('ElmoModel.h5')

probs = model_elmo.predict(testx, verbose=1)
print('lenght of probs : ' ,len(probs))
  #predicted_classes = probs.argmax(axis=-1)
predicted_classes=[0 if i < 0.5 else 1 for i in probs]
  #predicted_classes = attention_bilstm_model.predict_classes(X_test_initial, verbose=1)
  #print(predicted_classes)

print(classification_report(y_test, predicted_classes))

860/860 [==============================] - 7s 9ms/step
elmo Test Loss: 0.4640
elmo Test Accuracy: 0.7860
elmo Test f1 score: 0.4968
elmo  Test Precision: 0.7331
elmo BiRNN Test Recall: 0.4123
860/860 [==============================] - 7s 8ms/step
lenght of probs :  860
              precision    recall  f1-score   support

           0       0.80      0.93      0.86       620
           1       0.70      0.41      0.52       240

    accuracy                           0.79       860
   macro avg       0.75      0.67      0.69       860
weighted avg       0.77      0.79      0.77       860



In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,predicted_classes)

array([[578,  42],
       [142,  98]])